In [2]:
# 编写一个斐波拉切C扩展函数
# python中的实现
def fibonacci(n):
    if n < 2:
        return 1
    else:
        return fibonacci(n - 1) + fibonacci(n - 2)

In [1]:
# 纯C扩展 - 详见chapter4文件夹
# 构建指令：pip install -e <file folder>
from chapter4 import fibonacci
help(fibonacci.fibonacci)

Help on built-in function fibonacci in module chapter4.fibonacci:

fibonacci(...)
    fibonacci(n): Return nth Fibonacci sequence numbercomputed recursively



In [4]:
[fibonacci.fibonacci(n) for n in range(10)]

[1, 1, 2, 3, 5, 8, 13, 21, 34, 55]

In [2]:
fibonacci.fibonacci(-1)

ValueError: n must not be less than 0